# Flow chart 層級======

---------
1. TABLE NAMES (1:N)
2. FROM - 完整 join 表 (1)
3. WHER - 完整篩選條件列表(1)
4. GROUP BY - 完整group by 列表 (1)
5. ORDER BY - 完整order by 列表 (1)
6. limit - 數字 (1)
7. SELECT - 選取欄位列表 (1) 
8. CREATE TABLE NAME

# 子查詢取代KEY NAME Function
### 1.紀錄各項查詢LEVEL
- subq_counts : "_" 次數決定子查詢所在等級(0:主查詢、1:1st子查詢、2:2nd子查詢)
- subq_values : 取得子查詢列表的 sql表示
- subq_keys : 取得子查詢列表的 KEY標註 表示 ("MAIN", "MAIN_SELECT1", ....)

### 2. 兩迴圈分別檢視每一個 subq_values與下一層子查詢是否需要置換
- this_level : subq_counts[idx] ， 子查詢所在等級
- next_level : this_level + 1 ，子查詢下一層等級
- next_level_check : 觀察兩件事
 - 是否為下一層：subq_counts[idx_2nd]判斷是否等於 next level
 - 上下層等級是否相關 : ex: MAIN 與 MAIN_SELECT1相關、
   但MAIN_WHERE與MAIN_SELECT_1不相關
   
### 3. 置換KEY NAME 的VALUE by 第二層的KEYNAME，並加上SUBQ.作為子查詢標註
第二層KEYNAME : MAIN_FROM1、MAIN_WHERE1、MAIN_FROM1_FROM1....
- 範例：(subquery) → (MAIN_FROM1) → (SUBQ.MAIN_FROM1)
- 置換括號：(SUBQ.MAIN_FROM1) → SUBQ.MAIN_FROM1